## Assignment 3 

#### Instructions
Perform an analysis of the dataset(s) used in Homework #2 using the SVM algorithm. Compare the results with the results from previous homework.
Homework #3

- Read the following articles:
    - https://www.hindawi.com/journals/complexity/2021/5550344/
    - https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8137961/
- Search for academic content (at least 3 articles) that compare the use of decision trees vs SVMs in your current area of expertise.
- Perform an analysis of the dataset used in Homework #2 using the SVM algorithm.
- Compare the results with the results from previous homework.
- Answer questions, such as:
  - Which algorithm is recommended to get more accurate results?
  - Is it better for classification or regression scenarios?
  - Do you agree with the recommendations?
  - Why?

#### Format
 - Essay (minimum 500 word document) Write a short essay explaining your selection of algorithms and how they relate to the data and what you are trying to do.
 - Analysis using R or Python (submit code + errors + analysis as notebook or copy/paste to document).Include analysis R (or Python) code.

### Two Assigned Articles 

##### Decision Tree Ensembles to Predict Coronavirus Disease 2019 Infection: A Comparative Study (https://www.hindawi.com/journals/complexity/2021/5550344/)

- This paper looked at COvid-19 in patients and attempted to use decision tree modeling on lab results and patient age, with imbalanced outcome considerations, for properly predicting COVID diagnosis in patients. There were 5644 total patients in the original data, with 600 final patients being kept in the study using about 18 different lab biomarker measurements. Of the 600 patents used, 520 were negative and 80 were positive. So about a 13% positivity rate. Data was imbalanced so RUS and SMOTE were used, and the multiple types of Classifier methods were used to obtain results. Overall the best modeling technique was random forest having highest accuracy. with RUSBagging, Balanced Random ForestBalanced, and XGBoost with RUS doing well too.

##### A novel approach to predict COVID-19 using support vector machine (https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8137961/)

- This paper predicts COVID-19 status using a support vector machine modeling on individual's symptoms. The dataset was composrted of 200 individual's data with 8 different symtopms": temperature, breathing rate, hypertension, stage heart beat rate, acute respiratory distress syndrome, chest pain, heart disease and cough with sputum. The modeling attempted to categorize inidivduals into three differewnt classes: not infected, mildly infected, and severely infected. They used a linear SVM model and a 70/30 training/test split. THe modeling yielded The modeling yielded an overall accuracy of 87 percent, with class results showing the severely infected class with a precision  of 0.94, recall of 1.00, and f1 of 0.97. The the not infected and mildly infected classes were lower. SVM performed best on this dataset when compared to k-nearest neighbors,naive Bayes, random forest, and AdaBoost.

### Three Acadmic Researched Articles 

In [ ]:
##### 1) A comparative study of forecasting corporate credit ratings using neural networks, support vector machines, and decision trees 

##### 2) 
##### 3) 

### Performing SVM on Assignment 2 Data 

In [12]:
import pandas as pd
from pandas.plotting import scatter_matrix
import numpy as np
import json
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV, ParameterGrid
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import (average_precision_score, roc_auc_score,
                             classification_report, confusion_matrix,
                             PrecisionRecallDisplay)
from sklearn.preprocessing import StandardScaler 

#### Assignment 2 Replication for the SVM training and Analysis

In [2]:
### Pulling in the data from Assignment 1 
df = pd.read_csv("../Assignment2/raw_df_dropped.csv")

In [3]:
print(df.columns) # No duration because dropped in assignment one. (Data Leakage) 

## Encoding the y var for (1/0) | (yes/no)
df['y'] = df['y'].map({'no':0, 'yes':1})

## Parsing the 'y' from the x vars 
X = df.drop(columns=['y'])
y = df['y']

## Splitting the test and train sets for future experiments and modeling
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Confirming no nulls; no additional imputation needed because of Assignment 1 cleaning etc. 
# print(X_train.info())
# print(y_train.info())
# print(X_test.info())
# print(y_test.info())

Index(['age', 'job', 'marital', 'education', 'default', 'balance', 'housing',
       'loan', 'contact', 'day_of_week', 'month', 'campaign', 'previous',
       'poutcome', 'y', 'previously_contacted'],
      dtype='object')


In [4]:
# Getting lists of the x vars and their types For encoding pre-modeling 
num_cols = X_train.select_dtypes(include=['number']).columns.tolist()
print(num_cols)
cat_cols = [c for c in X_train.columns if c not in num_cols]
print(cat_cols)

['age', 'balance', 'day_of_week', 'campaign', 'previous']
['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome', 'previously_contacted']


In [5]:
X_train_encoded = pd.get_dummies(X_train, columns=cat_cols, drop_first=False)
X_test_encoded  = pd.get_dummies(X_test,  columns=cat_cols, drop_first=False)

In [6]:
print([i for i in X_train_encoded.columns if i not in X_test_encoded.columns])
print([i for i in X_test_encoded.columns if i not in X_train_encoded.columns])
## Columns are identical, no missing categories in each df so no need for alignment 

[]
[]


#### End Assignment Two Replica Code. Beginning of Assignment 3 Changes for SVM modeling

In [7]:
## Scaling the numeric data
scaler = StandardScaler()
X_train_scaled = X_train_encoded.copy()
X_test_scaled  = X_test_encoded.copy()
X_train_scaled[num_cols] = scaler.fit_transform(X_train_encoded[num_cols])
X_test_scaled[num_cols]  = scaler.transform(X_test_encoded[num_cols])

##  looking  at scaled vals sample 
X_train_scaled[num_cols].describe().T.head()

,count,mean,std,min,25%,50%,75%,max
age,35536.0,-2.975258e-16,1.000014,-2.149123,-0.738651,-0.174462,0.671821,5.091300
balance,35536.0,1.829544e-17,1.000014,-2.664600,-0.417440,-0.295665,0.018353,32.722977
day_of_week,35536.0,-4.618848e-17,1.000014,-1.776109,-0.935933,0.024268,0.624393,1.824645
campaign,35536.0,7.058239e-17,1.000014,-0.571367,-0.571367,-0.244724,0.081918,18.047243
previous,35536.0,-3.739067e-17,1.000014,-0.245294,-0.245294,-0.245294,-0.245294,113.211503


In [8]:
## Prepping Result Holding Vals
THRESH = 0.5
running_results = {}
rows = [] 

In [9]:
#Setting up the first linar SVM model
lin_base = LinearSVC(class_weight="balanced", random_state=42)
lin_cal  = CalibratedClassifierCV(lin_base, method="sigmoid", cv=5)

#Fitting the model
lin_cal.fit(X_train_scaled, y_train)

## Predicting values
lin_probs = lin_cal.predict_proba(X_test_scaled)[:, 1]

## adding results to the running dict.
lin_y_pred = (lin_probs >= THRESH).astype(int)

# what C did you use for LinearSVC? (default is 1.0 if you didn't set it)
lin_C = getattr(lin_base, "C", 1.0)
lin_params = {"C": lin_C,   "gamma": "N/A"}

running_results["svm_linear_cal"] = {"prob": lin_probs, "y_pred": lin_y_pred, "params": lin_params}

In [ ]:
## RBF SVM : curved boiundaries, imbalanced y values 
rbf = SVC(kernel="rbf", class_weight="balanced", probability=True, random_state=42)

## Trying different settings for liencncy and finding neighbors
param_grid = {
    "C":     [0.1, 0.25, 0.75, 1, 2, 3, 5, 10],
    "gamma": ["scale", 0.1, 0.06, 0.04, 0.03, 0.02, 0.01]
}

for params in ParameterGrid(param_grid):
    C = params["C"]; gamma = params["gamma"]
    key = f"svm_rbf_C{C}_g{gamma}"
    
    print(key)
    # 1) fit this combo
    clf = SVC(kernel="rbf", class_weight="balanced", probability=True,
              random_state=42, C=C, gamma=gamma)
    clf.fit(X_train_scaled, y_train)

    # 2) test predictions
    probs = clf.predict_proba(X_test_scaled)[:, 1]
    y_pred = (probs >= THRESH).astype(int)

    # 3) store minimal artifacts (probabilities + hard preds) -> for your later aggregation
    running_results[key] = {"prob": probs, "y_pred": y_pred, "params": params}

svm_rbf_C0.1_gscale
svm_rbf_C0.1_g0.1
svm_rbf_C0.1_g0.06
svm_rbf_C0.1_g0.04
svm_rbf_C0.1_g0.03
svm_rbf_C0.1_g0.02


In [ ]:
## Adding the Results Metrics to the Rows Lists via Running Results DIct
for key, value in running_results.items():
    
    print(key)
    model_prob = value["prob"]
    y_pred = value["y_pred"]
    params = value["params"]
    C = params["C"]
    gamma = params["gamma"]
    
    classfctn_rpt = classification_report( y_test, y_pred, target_names=['no','yes'], output_dict=True )
    
    roc_auc = round(metrics.roc_auc_score(y_test, model_prob), 4)
    pr_auc  = round(metrics.average_precision_score(y_test, model_prob), 4)
    tn, fp, fn, tp = metrics.confusion_matrix(y_test, y_pred).ravel()

    rows.append({
        "model": key,
        "roc_auc": roc_auc,
        "pr_auc": pr_auc,
        "precision_pos": classfctn_rpt["yes"]["precision"],
        "recall_pos":    classfctn_rpt["yes"]["recall"],
        "f1_pos":        classfctn_rpt["yes"]["f1-score"],
        "precision_neg": classfctn_rpt["no"]["precision"],
        "recall_neg":    classfctn_rpt["no"]["recall"],
        "f1_neg":        classfctn_rpt["no"]["f1-score"],
        "macro_precision":   classfctn_rpt["macro avg"]["precision"],
        "macro_recall":      classfctn_rpt["macro avg"]["recall"],
        "macro_f1":          classfctn_rpt["macro avg"]["f1-score"],
        "weighted_precision": classfctn_rpt["weighted avg"]["precision"],
        "weighted_recall":    classfctn_rpt["weighted avg"]["recall"],
        "weighted_f1":        classfctn_rpt["weighted avg"]["f1-score"],
        "tn": tn, "fp": fp, "fn": fn, "tp": tp,
        "C": C, "gamma": gamma
    })

In [ ]:
# All SVM results 
svm_results =  pd.DataFrame(rows).round(4)
svm_results['Assignment']="Assignment 3"

#### Pulling in Top Performing models from Assignment 2 

In [15]:
assgn2_best = pd.read_csv("../Assignment2/Assignment2_top3_model_results.csv")
assgn2_best['Assignment']="Assignment 2"

,model,roc_auc,pr_auc,precision_pos,recall_pos,f1_pos,precision_neg,recall_neg,f1_neg,macro_precision,macro_recall,macro_f1,weighted_precision,weighted_recall,weighted_f1,tn,fp,fn,tp
0,rand_forest3,0.8017,0.4422,0.4509,0.5243,0.4848,0.9347,0.9143,0.9244,0.6928,0.7193,0.7046,0.8775,0.8682,0.8724,7162,671,500,551
1,rand_forest2,0.7958,0.4350,0.4971,0.4139,0.4517,0.9231,0.9438,0.9333,0.7101,0.6789,0.6925,0.8727,0.8811,0.8764,7393,440,616,435
2,adaboost1,0.7851,0.4289,0.6285,0.1931,0.2955,0.9009,0.9847,0.9410,0.7647,0.5889,0.6182,0.8687,0.8910,0.8646,7713,120,848,203
